<a href="https://colab.research.google.com/github/DineshKarnati/Topic-Modelling-LDA/blob/main/1_KmeansTfIdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from IPython.display import display
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import string
from nltk.corpus import stopwords
import json
import glob
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
train_df = pd.read_csv("/content/drive/MyDrive/Topic_Modelling_Dataset/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Topic_Modelling_Dataset/test.csv")

In [23]:
train_df.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [24]:
# train_df['document'] = train_df.apply(lambda row: (row['TITLE'], row['ABSTRACT']), axis=1) fpms tuple
train_df['document'] = train_df['TITLE'] + ' ' + train_df['ABSTRACT']
test_df['document'] = test_df['TITLE'] + ' ' + test_df['ABSTRACT']

In [25]:
train_df.head(1)

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,document
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps ...


In [26]:
!pip install contractions

In [27]:
import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
import contractions

def remove_stops(text, stops):
    text = re.sub(r"AC\/\d{1,4}\/\d{1,4}", "", text) #
    text = " ".join([contractions.fix(i) for i in text.split() if len(i) > 0 and "@" not in i])
    text = re.sub('[^.,$%-:a-zA-Z0-9 \n\.]', ' ', text)
    text = re.sub(' +', ' ', text)
    words = text.split()
    final = []
    for word in words:
        if word not in stops:
            final.append(word)
    final = " ".join(final)
    final = final.translate(str.maketrans("", "", string.punctuation))
    final = "".join([i for i in final if not i.isdigit()])
    while "  " in final:
        final = final.replace("  ", " ")
    return (final)


def clean_docs(docs):
    stops = stopwords.words("english")
    final = []
    for doc in docs:
        clean_doc = remove_stops(doc, stops)
        final.append(clean_doc)
    return (final)

In [29]:
train_df['document'] = train_df['document'].apply(lambda x: x.lower())
test_df['document'] = test_df['document'].apply(lambda x: x.lower())
clean_docs([train_df['document'][0]])


['reconstructing subjectspecific effect maps predictive models allow subjectspecific inference analyzing disease related alterations neuroimaging data given subjects data inference made two levels global ie identifiying condition presence subject local ie detecting condition effect individual measurement extracted subjects data global inference widely used local inference used form subjectspecific effect maps rarely used existing models often yield noisy detections composed dispersed isolated islands article propose reconstruction method named rsm improve subjectspecific detections predictive modeling approaches particular binary classifiers rsm specifically aims reduce noise due sampling error associated using finite sample examples train classifiers proposed method wrappertype algorithm used different binary classifiers diagnostic manner ie without information condition presence reconstruction posed maximumaposteriori problem prior model whose parameters estimated training data class

In [30]:
training_df = train_df.head(1000)

In [31]:
vectorizer = TfidfVectorizer(
    lowercase = True,
    max_features  =   100,
    max_df = 0.8, # i.e  words occured more than 80% will be ignored
    min_df = 5,  # integer value if word not occured more than n times in whole corpus than it will be ignored
    ngram_range = (1,3),
    stop_words = 'english'

)

In [33]:
doc_vectors = vectorizer.fit_transform(training_df['document'])
feature_names = vectorizer.get_feature_names_out()


In [36]:
feature_names[2]

dense = doc_vectors.todense()
dense_lst = dense.tolist()

In [37]:
lst_keywords = []

for each_desc in dense_lst:
  i = 0
  keywords = []
  for word in each_desc:
    if word > 0:
      keywords.append(feature_names[i])
    i+=1
  lst_keywords.append(keywords)

In [41]:
print(training_df['document'][0])
print("----")
lst_keywords[0]

reconstructing subject-specific effect maps   predictive models allow subject-specific inference when analyzing disease
related alterations in neuroimaging data. given a subject's data, inference can
be made at two levels: global, i.e. identifiying condition presence for the
subject, and local, i.e. detecting condition effect on each individual
measurement extracted from the subject's data. while global inference is widely
used, local inference, which can be used to form subject-specific effect maps,
is rarely used because existing models often yield noisy detections composed of
dispersed isolated islands. in this article, we propose a reconstruction
method, named rsm, to improve subject-specific detections of predictive
modeling approaches and in particular, binary classifiers. rsm specifically
aims to reduce noise due to sampling error associated with using a finite
sample of examples to train classifiers. the proposed method is a wrapper-type
algorithm that can be used with differen

['accuracy',
 'algorithm',
 'data',
 'demonstrate',
 'different',
 'finite',
 'given',
 'information',
 'local',
 'method',
 'model',
 'models',
 'non',
 'parameters',
 'particular',
 'problem',
 'propose',
 'proposed',
 'results',
 'state',
 'training',
 'used',
 'using']

# for all the documents will perform clustering technique to make cluster of keywords


In [45]:
k = 10
model = KMeans(n_clusters =k, init ="k-means++", max_iter=100, n_init=1)
model.fit(doc_vectors)
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

with open ("/content/drive/MyDrive/Topic_Modelling_Dataset/Tfidf_kmeans.txt", "w", encoding="utf-8") as f:
    for i in range(k):
        f.write(f"Cluster {i}")
        f.write("\n")
        for ind in order_centroids[i, :10]:
            f.write (' %s' % terms[ind],)
            f.write("\n")
        f.write("\n")
        f.write("\n")



In [48]:
# kmeans TFIDF confines each document to confine to one topic
# but according to basic knowledge each document can have multiple topics so we use LDA to multiple topics coming from LDA
